In [1]:
import tensorflow as tf
import numpy as np

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = np.loadtxt('./data.csv', delimiter=',',
                 unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [4]:
global_step = tf.Variable(0, trainable=False, name='global_step')

In [6]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=Y,
                                           logits=model))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step=global_step)

### 신경망 모델 학습

In [8]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

In [18]:
# ./model 디렉터리에 기존에 학습해둔 모델이 있으면 학습된 값들을 불러오고
# 아니면 변수를 새로 초기화 한다.

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from ./model/dnn.ckpy-4


In [21]:
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: %d, ' % sess.run(global_step),
         'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

Step: 5,  Cost: 1.138
Step: 6,  Cost: 1.063


In [15]:
saver.save(sess, './model/dnn.ckpy', global_step=global_step)

'./model/dnn.ckpy-4'

In [22]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)

print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))


예측값: [0 2 2 0 0 2]
실제값: [0 1 2 0 0 2]


In [23]:
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data,
                                                       Y: y_data}))

정확도: 83.33


## 텐서보드 사용하기

In [5]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

data = np.loadtxt('./data.csv', delimiter=',',
                 unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))
    tf.summary.histogram("Weights", W1)

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=Y,
                                               logits=model))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)
    
    tf.summary.scalar('cost', cost)

sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: %d, ' % sess.run(global_step), 
         'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))
    

saver.save(sess, './model/dnn.ckpt', global_step=global_step)

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))
    
    



Step: 1,  Cost: 0.810
Step: 2,  Cost: 0.784
Step: 3,  Cost: 0.760
Step: 4,  Cost: 0.739
Step: 5,  Cost: 0.722
Step: 6,  Cost: 0.709
Step: 7,  Cost: 0.696
Step: 8,  Cost: 0.684
Step: 9,  Cost: 0.672
Step: 10,  Cost: 0.662
Step: 11,  Cost: 0.652
Step: 12,  Cost: 0.643
Step: 13,  Cost: 0.635
Step: 14,  Cost: 0.627
Step: 15,  Cost: 0.619
Step: 16,  Cost: 0.613
Step: 17,  Cost: 0.607
Step: 18,  Cost: 0.601
Step: 19,  Cost: 0.597
Step: 20,  Cost: 0.592
Step: 21,  Cost: 0.588
Step: 22,  Cost: 0.584
Step: 23,  Cost: 0.580
Step: 24,  Cost: 0.577
Step: 25,  Cost: 0.574
Step: 26,  Cost: 0.571
Step: 27,  Cost: 0.569
Step: 28,  Cost: 0.567
Step: 29,  Cost: 0.565
Step: 30,  Cost: 0.563
Step: 31,  Cost: 0.561
Step: 32,  Cost: 0.560
Step: 33,  Cost: 0.559
Step: 34,  Cost: 0.558
Step: 35,  Cost: 0.557
Step: 36,  Cost: 0.556
Step: 37,  Cost: 0.556
Step: 38,  Cost: 0.555
Step: 39,  Cost: 0.554
Step: 40,  Cost: 0.554
Step: 41,  Cost: 0.554
Step: 42,  Cost: 0.553
Step: 43,  Cost: 0.553
Step: 44,  Cost: 0.5